In [1]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader
from qdrant_client import QdrantClient
import tqdm
from pathlib import Path
from fastembed.embedding import FlagEmbedding
from pydantic import BaseModel, Field
from typing import Union, List
import abc

In [2]:
class DirectoryConfig(BaseModel):
    dir: Union[str, Path]
    glob_pattern: str

class DirectoryDocumentLoader(abc.ABC):

    @abc.abstractmethod
    def load_docs(self, dir_config: DirectoryConfig):
        pass


class LangchainDirectoryDocumentLoader(BaseModel, DirectoryDocumentLoader):
    chunk_size: int = Field(default=512)
    chunk_overlap: int = Field(default=128)
    show_progress: bool = Field(default=True)

    def load_raw_docs(self, dir, glob_pattern):
        loader = DirectoryLoader(dir, glob=glob_pattern, show_progress=self.show_progress)
        docs = loader.load()
        return docs
    
    def chunk_docs(self, docs):
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size = self.chunk_size,
            chunk_overlap  = self.chunk_overlap,
            length_function = len,
            is_separator_regex = False,
        )
        chunked_docs = text_splitter.split_documents(docs)
        return [self._langchain_doc_to_dict_doc(doc) for doc in chunked_docs]

    @classmethod
    def _langchain_doc_to_dict_doc(cls, doc):
        return {"document": doc.page_content, "metadata": doc.metadata}
                 
    def load_docs(self, dir_config: DirectoryConfig):
        raw_docs = self.load_raw_docs(dir_config.dir, dir_config.glob_pattern)
        return self.chunk_docs(raw_docs)

In [3]:
class QdrantIndexer(BaseModel):
    document_loader: DirectoryDocumentLoader
    qdrant_client: QdrantClient
    collection_name: str

    def index_docs_from_directory(self, dir_config: DirectoryConfig):
        docs = self.document_loader.load_docs(dir_config)
        self.index_docs(self.qdrant_client, docs, self.collection_name)
    
    class Config:
        arbitrary_types_allowed = True

    @classmethod
    def index_docs(cls, qdrant_client, docs: List[dict], collection_name: str):
        doc_contents = [doc["document"] for doc in docs]
        metadatas = [doc["metadata"] for doc in docs]
        qdrant_client.add(
            collection_name=collection_name,
            documents=doc_contents,
            metadata=metadatas,
            ids=range(len(docs)),
            batch_size=64,
            parallel=6
        )

    class Config:
        arbitrary_types_allowed = True

In [4]:
loader = LangchainDirectoryDocumentLoader()

In [5]:
#lc_docs = loader.load_docs(dir=Path("~/Projects/forks/dspy").expanduser(),  glob_pattern="**/*.py")

In [7]:
collection_name = "org"
qdrant_path = Path(f"~/.cache/qdrant/{collection_name}").expanduser()
client = QdrantClient(path=qdrant_path)

In [8]:
indexer = QdrantIndexer(document_loader=loader, qdrant_client=client, collection_name=collection_name)

In [9]:
org_dir = Path("~/Projects/org").expanduser()

In [10]:
%%time
indexer.index_docs_from_directory(dir_config=DirectoryConfig(dir=org_dir, glob_pattern="**/*.org"))

  3%|████▋                                                                                                                                                                     | 29/1048 [00:03<01:17, 13.12it/s][WARNING] Could not convert TeX math 'p(x) =  \int_{0}^x (\frac{x}{\beta})^{\alpha-1} e^{-\frac{x}{\beta}}dx', rendering as TeX
[WARNING] Could not convert TeX math '\Gamma(\frac{r}{2}, 2)', rendering as TeX

  7%|███████████▋                                                                                                                                                              | 72/1048 [00:07<01:11, 13.61it/s][WARNING] Ignoring unknown Org export option: tex:
[WARNING] Could not convert TeX math 'T_x \mathbb{H}^{N,K} = \{v \in \mathbb{R^{d+1}} | <v,x>_L = 0}', rendering as TeX:
  d+1}} | <v,x>_L = 0}
                     ^
  unexpected '}'
  expecting "\\bangle", "\\brace", "\\brack", "\\choose", "\\displaystyle", "{", letter, digit, ".", "\\mbox", "\\text", "\\textbf", "\\te

CPU times: user 23.3 s, sys: 2.7 s, total: 26 s
Wall time: 3min 26s


In [11]:
search_result = indexer.qdrant_client.query(
    collection_name="org",
    query_text="How to index documents with llamaindex?"
)
print(search_result)

[QueryResponse(id=1954, embedding=None, metadata={'document': '"""\n\nindex knows searchable documents but does not need to know actual documents\n\n"""\n\ndef index_documents(index: Index[RetrievalContextType], documents: Collection[RetrievalContextType, Document[RetrievalContextType]]):\n\npass\n\nQA DSL\n\n???', 'source': '/home/kuba/Projects/org/roam/20230523164215-qa_dsl.org'}, document='"""\n\nindex knows searchable documents but does not need to know actual documents\n\n"""\n\ndef index_documents(index: Index[RetrievalContextType], documents: Collection[RetrievalContextType, Document[RetrievalContextType]]):\n\npass\n\nQA DSL\n\n???', score=0.8762011719995155), QueryResponse(id=3435, embedding=None, metadata={'document': 'llama index response synthesis', 'source': '/home/kuba/Projects/org/roam/20230319113823-llm_response_synthesis.org'}, document='llama index response synthesis', score=0.8728227321555979), QueryResponse(id=3370, embedding=None, metadata={'document': 'task="text-